In [4]:
!pip3 install numpy

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [6]:
import pandas as pd
import numpy as np
import os
# import translate

- let's throw out the english data
- not bothering w translation to english

# Dataset 1a: German → English Argumentation Reviews (Thiemo)

1. Train a German GloVE model on these embeddings from column 'review'
2. WEAT Analysis
3. Word Co-occurence analysis

In [2]:
!pip install nltk

In [14]:
import nltk
from nltk.corpus import stopwords

german_stop_words = stopwords.words('german')
eng_stop_words = stopwords.words('english')

In [72]:
data_path = 'background/'
student_reviews_data = pd.read_excel(data_path + '201901_Studenten Reviews BI2 full.xlsx', header=1, index_col=0)
student_reviews_data

,itpa (Learning Unit),year,authorID (Ersteller des Reviews),reviewerID (Wer wurde gereviewt),reviewNumFromChange,review,Q_completed_task,Q_constructive_suggestions,Q_critical_aspects,Q_easy_understand,Q_high_quality,Q_highlights_weaknesses_strengths,Q_task_related,Q_was_helpful,review_list
0,1,2016,dominik.geissler@student.unisg.ch,boletin.asani@unisg.ch,1,Geschäftsidee HC19 Die Idee eine Art MetaPlat...,2,6,6,4,6,4,4,6,geschäftsidee hc19 die idee eine art metaplatt...
1,1,2016,dominik.geissler@student.unisg.ch,arne.meeuw@unisg.ch,3,Feedback zur Beschreibung / AllgemeinKonzept g...,2,7,6,4,6,4,4,6,feedback zur beschreibung allgemeinkonzept gef...
2,1,2016,dominik.geissler@student.unisg.ch,patrik.brueggemann@student.unisg.ch,2,1 Eine zentrale App für das Buchen von Tickets...,2,5,5,4,5,4,4,5,1 eine zentrale app für da buchen von tickets ...
3,2,2016,marc.burgunder@student.unisg.ch,clarajohanna.schruff@student.unisg.ch,2,"Die Ausführungen sind sehr gut, detailliert un...",2,3,3,4,2,4,4,3,die ausführungen sind sehr gut detailliert und...
4,2,2016,marc.burgunder@student.unisg.ch,fabian.rados@student.unisg.ch,1,Worin sehen Sie die Stärken der Lösungdes Komm...,2,5,4,4,5,4,4,5,worin sehen sie die stärken der lösungdes komm...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5157,6,2017,felix.meindl@student.unisg.ch,delia.wyss@student.unisg.ch,3,Worin sehen Sie die Stärken der Lösungdes Komm...,2,5,6,6,5,5,6,6,worin sehen sie die stärken der lösungdes komm...
5158,6,2017,felix.meindl@student.unisg.ch,gabriel.barmettler@student.unisg.ch,1,"Könnte Teils etwas ausführlicher sein, enthält...",2,4,6,5,5,3,5,6,könnte teils etwas ausführlicher sein enthält ...
5159,6,2017,terence.la@student.unisg.ch,bastian.zarskebueno@student.unisg.ch,3,Liebe/r Kommilitone/inEs war sehr interessant ...,2,4,6,5,5,5,5,4,liebe/r kommilitone/ines war sehr interessant ...
5160,6,2017,terence.la@student.unisg.ch,marco.sansoni@student.unisg.ch,2,Du hast mit deinem Geschäftsmodellbeschrieb ei...,2,3,5,5,5,5,3,4,du hast mit deinem geschäftsmodellbeschrieb ei...


In [75]:
student_reviews_data.iloc[11]

itpa (Learning Unit)                                                                 1
year                                                                              2016
authorID (Ersteller des Reviews)                        valentin.kahn@student.unisg.ch
reviewerID (Wer wurde gereviewt)                    mirjam.zimmermann@student.unisg.ch
reviewNumFromChange                                                                  3
review                               Das Geschäftsmodell von Nutreno erscheint mir ...
Q_completed_task                                                                     2
Q_constructive_suggestions                                                           7
Q_critical_aspects                                                                   7
Q_easy_understand                                                                    4
Q_high_quality                                                                       7
Q_highlights_weaknesses_strengths          

In [16]:
import re
import string

def stop_word_removal(x):
    token = x.split()
    cleaned = ' '.join([w for w in token if not w in german_stop_words])
    return cleaned.replace('.', '\n')

def eng_stop_word_removal(x):
    token = x.split()
    cleaned = ' '.join([w for w in token if not w in eng_stop_words])
    return cleaned.replace('.', '\n')

def clean_text(text):
    # remove numbers
    text_nonum = re.sub(r'\d+', '', text)
    # remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum]) 
    # substitute multiple whitespace with single whitespace
    # Also, removes leading and trailing whitespaces
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace

student_reviews_data['review'] = student_reviews_data['review'].apply(stop_word_removal)
student_reviews_data['review'] = student_reviews_data['review'].apply(clean_text)
full_review = '\n'.join(student_reviews_data['review'])
with open("reviews_glove.txt", "w") as text_file:
    text_file.write(full_review)

In [61]:
data_path = 'background/data/'
all_files_roman = os.listdir(data_path)
roman_reviews = []
for file in all_files_roman:
    data_file = pd.read_csv(data_path + file)
    roman_reviews.extend(data_file['review'])

In [62]:
len(roman_reviews)

9156

In [65]:
roman_reviews = pd.Series(roman_reviews)

In [66]:
roman_reviews = roman_reviews.apply(stop_word_removal)
roman_reviews = roman_reviews.apply(clean_text)
full_review = '\n'.join(roman_reviews)
with open("roman_reviews_glove.txt", "w") as text_file:
    text_file.write(full_review)

# Dataset 1b: German Argumentation Reviews (Thiemo)

0. Translate text to English
1. Train an English GloVE model on these embeddings from column 'review'
2. WEAT Analysis
3. Word Co-occurence analysis

benefits: direct comparison to baselines from the original paper  
cons: how much bias can be attributed to the content, and how much to the translator?

In [6]:
import deep_translator
from deep_translator import GoogleTranslator

translate_from_scratch = False

if translate_from_scratch:
    translator = GoogleTranslator(source='de', target='en')

    def translate_5000(text):
        characters = len(text)
        print(characters > 3000)
        translations = []
        for i in np.arange(3000, characters, 3000):
            translation = translator.translate(text[i-3000:i])
            translations.append(translation)
        translations.append(translator.translate(text[:-characters%3000]))
        return ''.join(translations)

    student_reviews_data['review'] = student_reviews_data['review'].apply(lambda k: translate_5000(k))

In [7]:
if translate_from_scratch:
    student_reviews_data.to_csv('english_translated_student_reviews.csv')

# Dataset 2: German Annotated Essays

1. Train an German GloVE model on these embeddings from student essays
2. WEAT Analysis
3. Word Co-occurence analysis
4. Analyze using annotations

In [8]:
data_path = 'background/Coprus Coling/Annotations/'
coling_essay_files = [file for file in os.listdir(data_path) if ".txt" in file]
essays = {}
for essay_file in coling_essay_files:
    with open(data_path+essay_file, "r") as myfile:
        data = myfile.read().splitlines()
        data_cleaned = clean_text(stop_word_removal(data[1]))
        essays[int(essay_file.split('.txt')[0])] = ''.join(data[1].values())

In [9]:
all_essays = '\n'.join(essays.values())

with open("essays_glove.txt", "w") as text_file:
    text_file.write(all_essays)

# Dataset 3: English Annotated Essays

1. Train an English GloVE model on these embeddings from student essays
2. WEAT Analysis
3. Word Co-occurence analysis
4. Analyze using annotations 

In [17]:
data_path = 'background/ArgumentAnnotatedEssays-2.0/brat-project-final/'
eng_essay_files = [file for file in os.listdir(data_path) if ".txt" in file]
essays = {}
for essay_file in eng_essay_files:
    with open(data_path+essay_file, "r") as myfile:
        data = myfile.read()
        data_cleaned = eng_stop_word_removal(clean_text(data))
        essays[int(essay_file.split('.txt')[0].split('essay')[1])] = data_cleaned
essays

{22: "young people go university persistent deliberation happening whether desirable youth go university\n regard development country, people believe essential young students get high quality education universities, others think one causes increase rate unemployment\n essay, analyze views first order come fool-proof decision\n begin with, advocate impracticability university several reasons\n point view, fact numerous job opportunities certification university graduation, mechanics, sales persons, engineers, and, furthermore, labors graduates short supply\n worse, labor market highly educated students competitive get job finishing studying\n regard loss national productivity young people spend much time searching high salary jobs, even sheer time-consuming\n however, side coin voices opposition, saying universities provide skills careers, also academic knowledge human race, bio science, politics, medicine\n academic values necessary progress world\n importantly, many researches ivory-t

In [18]:
all_essays = '\n'.join(essays.values())
with open("eng_essays_glove.txt", "w") as text_file:
    text_file.write(all_essays)

In [19]:
len(essays.values())

402

# Generate GloVE models

In [30]:
%cd GloVE
!bash essays.sh
!bash eng_essays.sh
!bash reviews.sh

/Users/vinitra/Desktop/argmining20-social-bias-argumentation/GloVe
mkdir -p build

$ build/vocab_count -min-count 5 -verbose 2 < ../essays_glove.txt > essay_vocab.txt
BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.Processed 241611 tokens.
Counted 29697 unique words.
Truncating vocabulary at min count 5.
Using vocabulary of size 4469.

$ build/cooccur -memory 4.0 -vocab-file essay_vocab.txt -verbose 2 -window-size 15 < ../essays_glove.txt > cooccurrence.bin
COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 13752509
overflow length: 38028356
Reading vocab from file "essay_vocab.txt"...loaded 4469 words.
Building lookup table...table contains 18882279 elements.
Processing token: 0100000200000Processed 241611 tokens.
Writing cooccurrences to disk.......2 files in total.
Merging cooccurrence files: processed 0 lines.100000 lines.200000 lines.300000 lines.400000 lines.500000 lines.600000 lines.700000 lines.800000 lines.900000 lines.1000000 lines.11000

done.
vector size: 300
vocab size: 2492
x_max: 10.000000
alpha: 0.750000
04/19/22 - 03:27.46PM, iter: 001, cost: 0.059008
04/19/22 - 03:27.46PM, iter: 002, cost: 0.052082
04/19/22 - 03:27.46PM, iter: 003, cost: 0.050889
04/19/22 - 03:27.46PM, iter: 004, cost: 0.050307
04/19/22 - 03:27.46PM, iter: 005, cost: 0.049632
04/19/22 - 03:27.46PM, iter: 006, cost: 0.048301
04/19/22 - 03:27.46PM, iter: 007, cost: 0.046985
04/19/22 - 03:27.47PM, iter: 008, cost: 0.046077
04/19/22 - 03:27.47PM, iter: 009, cost: 0.045308
04/19/22 - 03:27.47PM, iter: 010, cost: 0.044386
04/19/22 - 03:27.47PM, iter: 011, cost: 0.043229
04/19/22 - 03:27.47PM, iter: 012, cost: 0.041859
04/19/22 - 03:27.47PM, iter: 013, cost: 0.040332
04/19/22 - 03:27.47PM, iter: 014, cost: 0.038673
04/19/22 - 03:27.47PM, iter: 015, cost: 0.036974
04/19/22 - 03:27.47PM, iter: 016, cost: 0.035314
04/19/22 - 03:27.48PM, iter: 017, cost: 0.033697
04/19/22 - 03:27.48PM, iter: 018, cost: 0.032127
04/19/22 - 03:27.48PM, iter: 019, cost: 0.030

04/19/22 - 03:28.29PM, iter: 011, cost: 0.027555
04/19/22 - 03:28.31PM, iter: 012, cost: 0.025893
04/19/22 - 03:28.34PM, iter: 013, cost: 0.024299
04/19/22 - 03:28.37PM, iter: 014, cost: 0.022788
04/19/22 - 03:28.39PM, iter: 015, cost: 0.021359
04/19/22 - 03:28.42PM, iter: 016, cost: 0.020019
04/19/22 - 03:28.45PM, iter: 017, cost: 0.018779
04/19/22 - 03:28.47PM, iter: 018, cost: 0.017631
04/19/22 - 03:28.50PM, iter: 019, cost: 0.016586
04/19/22 - 03:28.53PM, iter: 020, cost: 0.015627
04/19/22 - 03:28.55PM, iter: 021, cost: 0.014772
04/19/22 - 03:28.58PM, iter: 022, cost: 0.014000
04/19/22 - 03:29.01PM, iter: 023, cost: 0.013286
04/19/22 - 03:29.03PM, iter: 024, cost: 0.012649
04/19/22 - 03:29.06PM, iter: 025, cost: 0.012077
04/19/22 - 03:29.09PM, iter: 026, cost: 0.011563
04/19/22 - 03:29.11PM, iter: 027, cost: 0.011102
04/19/22 - 03:29.14PM, iter: 028, cost: 0.010691
04/19/22 - 03:29.17PM, iter: 029, cost: 0.010304
04/19/22 - 03:29.19PM, iter: 030, cost: 0.009961
04/19/22 - 03:29.22P

# WEAT Co-occurence Analysis

In [31]:
import spacy

In [43]:
# !python -m spacy download en_core_web_sm
# !python -m spacy download de_core_news_sm
%cd ..

/Users/vinitra/Desktop/argmining20-social-bias-argumentation


In [44]:
!python weat_cooccurrence_analysis.py \
    --data "eng_essays_glove.txt" \
    --output "output/weat_cooccurrence_analysis" \
    --processing_cores 7 \
    --tests 1 2 3 4 5 6 7 8 9 10

INFO:2022-04-19 16:30:18:Please make sure that your input texts are whitespace separated tokens. The script might not work correctly otherwise.
INFO:2022-04-19 16:30:18:Generating target-association test pairs...
INFO:2022-04-19 16:30:18:Extracting candidate posts...
100%|███████████████████████████████████| 12538/12538 [00:09<00:00, 1299.91it/s]
INFO:2022-04-19 16:30:28:Splitting candidate posts into sentences...
100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 1310.72it/s]
INFO:2022-04-19 16:30:33:Calculating sentence-based cooccurrences...
12538it [00:01, 9862.77it/s] 
INFO:2022-04-19 16:30:35:Unpacking cooccurrences into a more useful format...
INFO:2022-04-19 16:30:35:Exporting results to disk at output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220419163035.json.
Done.


In [54]:
!python accumulate_cooccurrence_counts.py \
    --input "output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220419163035.json" \
    --weat_lexicons "sbeval/tests/weat_tests.json"

INFO:2022-04-19 16:36:19:Printing accumulated results for corpus 'eng_essays_glove.txt'.


Current test: test1
---------------------------------------------
X Flowers
Association-A:Association-B
0:0
Y insects
Association-A:Association-B
0:0


Current test: test2
---------------------------------------------
X Instruments
Association-A:Association-B
0:0
Y weapons
Association-A:Association-B
0:0


Current test: test3
---------------------------------------------
X European-American
Association-A:Association-B
0:0
Y African-American names
Association-A:Association-B
0:0


Current test: test4
---------------------------------------------
X European-American
Association-A:Association-B
0:0
Y African-American names
Association-A:Association-B
0:0


Current test: test5
---------------------------------------------
X European-American
Association-A:Association-B
0:0
Y African-American names
Association-A:Association-B
0:0


Current test: test6
---------------------------------------------
X Ma

In [46]:
!python weat_cooccurrence_analysis_german.py \
    --data "reviews_glove.txt" \
    --output "output/weat_cooccurrence_analysis" \
    --processing_cores 7 \
    --tests 1 2 3 4 5 6 7 8 9 10

INFO:2022-04-19 16:30:35:Please make sure that your input texts are whitespace separated tokens. The script might not work correctly otherwise.
INFO:2022-04-19 16:30:36:Generating target-association test pairs...
INFO:2022-04-19 16:30:36:Extracting candidate posts...
100%|██████████████████████████████████████| 7557/7557 [00:50<00:00, 150.06it/s]
INFO:2022-04-19 16:31:27:Splitting candidate posts into sentences...
100%|████████████████████████████████████████| 31/31 [00:00<00:00, 39689.69it/s]
INFO:2022-04-19 16:31:41:Calculating sentence-based cooccurrences...
7557it [00:01, 3934.20it/s]
INFO:2022-04-19 16:31:43:Unpacking cooccurrences into a more useful format...
INFO:2022-04-19 16:31:43:Exporting results to disk at output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220419163143.json.
Done.


In [68]:
!python accumulate_cooccurrence_counts.py \
    --input "output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220419163143.json" \
    --weat_lexicons "sbeval/tests/weat_tests_german.json"

INFO:2022-04-19 16:57:29:Printing accumulated results for corpus 'reviews_glove.txt'.


Current test: test1
---------------------------------------------
X Blumen
Association-A:Association-B
0:0
Y Insekten
Association-A:Association-B
0:0


Current test: test2
---------------------------------------------
X Instrumente
Association-A:Association-B
0:0
Y Waffen
Association-A:Association-B
0:0


Current test: test3
---------------------------------------------
X Europäisch-amerikanische
Association-A:Association-B
0:0
Y afro-amerikanische Namen
Association-A:Association-B
0:0


Current test: test4
---------------------------------------------
X Europäisch-amerikanische
Association-A:Association-B
0:0
Y afro-amerikanische Namen
Association-A:Association-B
0:0


Current test: test5
---------------------------------------------
X Europäisch-amerikanische
Association-A:Association-B
0:0
Y afro-amerikanische Namen
Association-A:Association-B
0:0


Current test: test6
---------------------------

In [67]:
!python weat_cooccurrence_analysis_german.py \
    --data "roman_reviews_glove.txt" \
    --output "output/weat_cooccurrence_analysis" \
    --processing_cores 7 \
    --tests 1 2 3 4 5 6 7 8 9 10

INFO:2022-04-19 16:55:08:Please make sure that your input texts are whitespace separated tokens. The script might not work correctly otherwise.
INFO:2022-04-19 16:55:08:Generating target-association test pairs...
INFO:2022-04-19 16:55:08:Extracting candidate posts...
100%|███████████████████████████████████████| 7557/7557 [01:53<00:00, 66.44it/s]
INFO:2022-04-19 16:57:02:Splitting candidate posts into sentences...
100%|████████████████████████████████████████| 59/59 [00:00<00:00, 51977.30it/s]
INFO:2022-04-19 16:57:17:Calculating sentence-based cooccurrences...
7557it [00:03, 2048.76it/s]
INFO:2022-04-19 16:57:21:Unpacking cooccurrences into a more useful format...
INFO:2022-04-19 16:57:21:Exporting results to disk at output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220419165721.json.
Done.


In [69]:
!python accumulate_cooccurrence_counts.py \
    --input "output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220419165721.json" \
    --weat_lexicons "sbeval/tests/weat_tests_german.json"

INFO:2022-04-19 16:57:35:Printing accumulated results for corpus 'roman_reviews_glove.txt'.


Current test: test1
---------------------------------------------
X Blumen
Association-A:Association-B
0:0
Y Insekten
Association-A:Association-B
0:0


Current test: test2
---------------------------------------------
X Instrumente
Association-A:Association-B
0:0
Y Waffen
Association-A:Association-B
0:0


Current test: test3
---------------------------------------------
X Europäisch-amerikanische
Association-A:Association-B
3:0
Y afro-amerikanische Namen
Association-A:Association-B
0:0


Current test: test4
---------------------------------------------
X Europäisch-amerikanische
Association-A:Association-B
3:0
Y afro-amerikanische Namen
Association-A:Association-B
0:0


Current test: test5
---------------------------------------------
X Europäisch-amerikanische
Association-A:Association-B
3:0
Y afro-amerikanische Namen
Association-A:Association-B
0:0


Current test: test6
---------------------

In [48]:
!python weat_cooccurrence_analysis_german.py \
    --data "essays_glove.txt" \
    --output "output/weat_cooccurrence_analysis" \
    --processing_cores 7 \
    --tests 1 2 3 4 5 6 7 8 9 10

INFO:2022-04-19 16:31:44:Please make sure that your input texts are whitespace separated tokens. The script might not work correctly otherwise.
INFO:2022-04-19 16:31:45:Generating target-association test pairs...
INFO:2022-04-19 16:31:45:Extracting candidate posts...
100%|██████████████████████████████████████| 7557/7557 [00:19<00:00, 388.96it/s]
INFO:2022-04-19 16:32:04:Splitting candidate posts into sentences...
100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 2933.08it/s]
INFO:2022-04-19 16:32:18:Calculating sentence-based cooccurrences...
7557it [00:01, 5410.62it/s]
INFO:2022-04-19 16:32:20:Unpacking cooccurrences into a more useful format...
INFO:2022-04-19 16:32:20:Exporting results to disk at output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220419163220.json.
Done.


In [1]:
!python accumulate_cooccurrence_counts.py \
    --input "output/weat_cooccurrence_analysis/weat-cooccurrence-analysis_results-20220419163220.json" \
    --weat_lexicons "sbeval/tests/weat_tests_german.json"

  File "accumulate_cooccurrence_counts.py", line 17
    logging.info(f"Printing accumulated results for corpus '{results['corpus']}'.")
                                                                                 ^
SyntaxError: invalid syntax


In [7]:
!bash convert_glove_to_word2vec_format.sh


\e================================================================================\e
\eConverting ./GloVe/glove_models_trained/eng_essay_vectors.txt...\e

\e================================================================================\e
\eConverting ./GloVe/glove_models_trained/essay_vectors.txt...\e

\e================================================================================\e
\eConverting ./GloVe/glove_models_trained/reviews_vectors.txt...\e

\e================================================================================\e
\eConverting ./GloVe/glove_models_trained/roman_reviews_vectors.txt...\e


In [18]:
!pip3 install gensim

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached gensim-4.1.2-cp39-cp39-macosx_11_0_arm64.whl
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no 

In [19]:
!python3 embedding_bias_evaluation.py \
    --embedding_model "GloVe/glove_models_trained/roman_reviews_vectors.txt" \
    --output "output/embedding_model_evaluation" \
    --lowercase

INFO:2022-04-22 16:10:43:Loading embedding model from disk.
INFO:2022-04-22 16:10:43:loading projection weights from GloVe/glove_models_trained/roman_reviews_vectors.txt
INFO:2022-04-22 16:10:44:KeyedVectors lifecycle event {'msg': 'loaded (18638, 300) matrix of type float32 from GloVe/glove_models_trained/roman_reviews_vectors.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-22T16:10:44.774416', 'gensim': '4.1.2', 'python': '3.9.12 (main, Mar 26 2022, 15:45:34) \n[Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-11.6-arm64-arm-64bit', 'event': 'load_word2vec_format'}
INFO:2022-04-22 16:10:44:Evaluating WEAT tests.
INFO:2022-04-22 16:10:44:Exporting results to disk at output/embedding_model_evaluation/embedding_bias_evaluation_results-20220422161044.json.
Done.


In [10]:
!python3 embedding_bias_evaluation.py \
    --embedding_model "GloVe/glove_models_trained/eng_essay_vectors.txt" \
    --output "output/embedding_model_evaluation" \
    --lowercase

Traceback (most recent call last):
  File "/Users/vinitra/Desktop/argmining20-social-bias-argumentation/embedding_bias_evaluation.py", line 9, in <module>
    from sbeval.weat_test import weat_score
  File "/Users/vinitra/Desktop/argmining20-social-bias-argumentation/sbeval/weat_test.py", line 13, in <module>
    from scipy.spatial.distance import cdist
  File "/opt/homebrew/lib/python3.9/site-packages/scipy/spatial/__init__.py", line 102, in <module>
    from ._kdtree import *
  File "/opt/homebrew/lib/python3.9/site-packages/scipy/spatial/_kdtree.py", line 5, in <module>
    from ._ckdtree import cKDTree, cKDTreeNode
  File "_ckdtree.pyx", line 1, in init scipy.spatial._ckdtree
ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject


In [21]:
!python3 embedding_bias_evaluation_german.py \
    --embedding_model "GloVe/glove_models_trained/essay_vectors.txt" \
    --output "output/embedding_model_evaluation" \
    --lowercase

INFO:2022-04-22 16:16:53:Loading embedding model from disk.
INFO:2022-04-22 16:16:53:loading projection weights from GloVe/glove_models_trained/essay_vectors.txt
INFO:2022-04-22 16:16:53:KeyedVectors lifecycle event {'msg': 'loaded (4472, 300) matrix of type float32 from GloVe/glove_models_trained/essay_vectors.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-22T16:16:53.721394', 'gensim': '4.1.2', 'python': '3.9.12 (main, Mar 26 2022, 15:45:34) \n[Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-11.6-arm64-arm-64bit', 'event': 'load_word2vec_format'}
INFO:2022-04-22 16:16:53:Evaluating WEAT tests.
INFO:2022-04-22 16:16:53:Exporting results to disk at output/embedding_model_evaluation/embedding_bias_evaluation_results-20220422161653.json.
Done.


In [53]:
!python embedding_bias_evaluation_german.py \
    --embedding_model "GloVe/glove_models_trained/reviews_vectors.txt" \
    --output "output/embedding_model_evaluation" \
    --lowercase

INFO:2022-04-19 16:32:23:Loading embedding model from disk.
INFO:2022-04-19 16:32:23:loading projection weights from GloVe/glove_models_trained/reviews_vectors.txt
INFO:2022-04-19 16:32:24:KeyedVectors lifecycle event {'msg': 'loaded (10769, 300) matrix of type float32 from GloVe/glove_models_trained/reviews_vectors.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2022-04-19T16:32:24.044840', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:24:02) \n[Clang 11.1.0 ]', 'platform': 'macOS-11.6-arm64-arm-64bit', 'event': 'load_word2vec_format'}
INFO:2022-04-19 16:32:24:Evaluating WEAT tests.
INFO:2022-04-19 16:32:24:Exporting results to disk at output/embedding_model_evaluation/embedding_bias_evaluation_results-20220419163224.json.
Done.
